In [1]:
import pandas as pd
import mysql.connector
from scipy.stats import ttest_ind
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Connecting to DataBase

In [206]:
db = mysql.connector.connect(
   host="localhost",
  user="root",
  password="Ben1204",
  database="Litopia")
mycursor = db.cursor()

# 2 : Impact of discount on Sales

In [240]:
sql_query = """
SELECT
    o.Quantity,
    o.discount,
    o.Price
FROM product AS p
INNER JOIN orders AS o ON p.ID = o.Product_ID;
"""
mycursor.execute(sql_query)

In [241]:
columns = [i[0] for i in mycursor.description]
result = mycursor.fetchall()
laptop_df= pd.DataFrame(result, columns=columns)


In [242]:
discount_df = laptop_df[(laptop_df['discount'] > 0) ]
without_discount_df = laptop_df[(laptop_df['discount'] == 0)]
discount_sample = discount_df.sample(n=100000,random_state=4)
without_discount_sample = without_discount_df.sample(n=100000,random_state=4)

In [234]:
#discount_sample=discount_sample[discount_sample['Quantity']>=2]
#without_discount_sample=without_discount_sample[without_discount_sample['Quantity']>=2]

In [249]:
print(without_discount_sample['Quantity'].sum(),discount_sample['Quantity'].sum())

110694 110606


## Total Quantity :
- With Discouint : 110606
- Without Discount: 110694

In [245]:
print(discount_df['Quantity'].mean(),without_discount_df['Quantity'].mean())

1.105427429675869 1.1064411244853478


## Avg Quantity : 
- With Discount :1.105427429675869 
- Without Discount : 1.1064411244853478

In [246]:
quantity_discount= discount_sample ['Quantity'].tolist()
quantity_without_discount = without_discount_sample['Quantity'].tolist()
quantity_discount = [int(item) for item in quantity_discount ]
quantity_without_discount = [int(item) for item in quantity_without_discount ]

In [247]:
t_statistic, p_value = ttest_ind(quantity_without_discount,quantity_discount, equal_var=True)
print("T-Statistic:", t_statistic)
print("P-Value:", p_value)

T-Statistic: 0.5408065211877862
P-Value: 0.5886415498384536


## T-Statistic:
- The t-statistic measures the difference between the means of the two groups (quantity with discounts and quantity without discounts) relative to the variation within each group. 
- A t-statistic of 0.5408 suggests that the difference in quantity is not significantly large.

## P-Value:
- The p-value represents the probability that you would observe the data you have if there were no real difference between the groups. In this case, a p-value of 0.5886 is relatively high. It means that there is a 58.86% chance of observing these results (or more extreme results) if there were no significant difference in the quantities between the two groups.

- Since the p-value is greater than a typical significance level of 0.05 (a commonly chosen value), you would fail to reject the null hypothesis. The null hypothesis (H0) in this context would be that there is no significant difference in the quantities of items sold with and without discounts.

# 3 : Impact of discount on profit 

- I used the profit percentage relative to the original price for better accuracy :(Profit/Total_price)*100 

In [257]:
sql_query = """
SELECT p.Model_Name, (o.Profit/o.Total_price)*100 as Profit, o.discount, o.Total_price
FROM product AS p
INNER JOIN orders AS o ON p.ID = o.Product_ID
WHERE o.Total_price > o.Profit 
"""
mycursor.execute(sql_query)


In [258]:
columns = [i[0] for i in mycursor.description]
result = mycursor.fetchall()
df= pd.DataFrame(result, columns=columns)


In [259]:
df_discount = df[df['discount']>0]
df_without_discount=df[df['discount']==0]
discount_sample = df_discount.sample(n=100000,random_state=4)
without_sample = df_without_discount.sample(n=100000,random_state=4)

In [260]:
profit_discount= discount_sample ['Profit'].tolist()
profit_without_discount =without_sample ['Profit'].tolist()
profit_discount = [float(item) for item in profit_discount ]
profit_without_discount = [float(item) for item in profit_without_discount ]

In [261]:
t_statistic, p_value = ttest_ind(profit_without_discount,profit_discount, equal_var=True)
print("T-Statistic:", t_statistic)
print("P-Value:", p_value)

T-Statistic: 1.8195569166129457
P-Value: 0.06882800207980423


 ## T-Statistic:
- The T-Statistic measures the difference between the means of the two groups (profit with discounts and profit without discounts).
- In your case, a T-Statistic of 0.3530577962510027 a negative T-statistic indicates that the sample mean of the first group (likely the profits without discounts) is lower than the sample mean of the second group (likely the profits with discounts).
## P-Value:
-  The P-Value is a measure of the probability of observing such extreme results (or more extreme) if there were no significant difference between the two groups.
- With a P-value of P-Value: 0.7240454838131373(which is greater than the typical significance level of 0.05), in this case, with a p-value of 0.7240454838131373, it is quite high, suggesting that the observed effect is not statistically significant at a common significance level of 0.05. 